In [1]:
from multiprocessing import Pool
from functools import partial
from contextlib import closing


import numpy as np
import pytesseract
from PIL import Image
import cv2
from matplotlib import pyplot as plt

# Importing the one of the image files (PNG in this case due to the lossless compression algorithm - we should read directly into Python from PDF though)

image = cv2.imread('../msk_RF1-0.png',cv2.IMREAD_GRAYSCALE)
#image = cv2.imread('comp.png',cv2.IMREAD_GRAYSCALE)

#image = cv2.imread('../msk_RF1-0.png')

#from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import imutils

In [2]:
#auxilliary function for contour identification    
def Operate(j,cnts):
    c=cnts[j]
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    return approx


def RecursiveSplitImageByContour(image, Depth="0"):
    ratio = image.shape[0] 
    orig = image.copy()
    #image = imutils.resize(image, height=1000)
 
    # convert the image to grayscale, blur it, and find edges
    # in the image
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(image, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    
    contours = []
    if edged is not None:

        # find the contours in the edged image, keeping only the
        # largest ones, and initialize the screen contour
        cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
    
        #parallel mapping of contouring function
        func = partial(Operate, cnts=cnts)
        with closing( Pool(processes = 8, maxtasksperchild=1)) as p:
            cl_map = p.map(func, range(len(cnts)))
        
        #minimal paragrapth size required (no further splitting implied)
        MinHeight = 50 #pixels
        
        # loop over the contours
        for c in cl_map:
            # approximate the contour 
            # if our approximated contour has four points, then we
            # can assume that we have found our screen
        
        
            if len(c) == 4:
                if abs(c[0][0][1]-c[2][0][1]) > MinHeight :
                    contours.append(c)
    
        #print number of contours
        #only the first contour is used anyway but it is important that it was found
        #a.k.a number of contour is greater then 0
        numberOfContour = len(contours)
        print( str(numberOfContour) +" contours found...")
    
    else:
        print("No edges identified")
    
    #recursion ancor (recursion is stopped when no contour is found)
    if numberOfContour == 0:
        cv2.imwrite('./ImageSplits/split_'+Depth+".png",image) 
        return
    
    #splitting image by defined contours
    splits = (image[:contours[0][0][0][1], : ],
              image[contours[0][0][0][1]:contours[0][2][0][1],: ],
              image[contours[0][2][0][1]::,  ])
        
    #counting the splits for further taxonomisation
    counter = 0
        
    #looping over splits
    for split in splits:
            
        #skipping tiny sections if height smaller then 15 pixels
        if split.shape[0] < 15:
            continue
            
        #increasing counter
        counter+=1
        #saving the file
        RecursiveSplitImageByContour(split, Depth+str(counter)+"_")
            

    return

In [3]:
RecursiveSplitImageByContour(image)

4 contours found...
0 contours found...
4 contours found...
5 contours found...
0 contours found...
0 contours found...
0 contours found...
0 contours found...
5 contours found...
2 contours found...
0 contours found...
0 contours found...
0 contours found...
5 contours found...
5 contours found...
0 contours found...
5 contours found...
0 contours found...
0 contours found...
2 contours found...
0 contours found...
5 contours found...
0 contours found...
2 contours found...
0 contours found...
0 contours found...
0 contours found...
